In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/march-machine-learning-mania-2025/Conferences.csv
/kaggle/input/march-machine-learning-mania-2025/SeedBenchmarkStage1.csv
/kaggle/input/march-machine-learning-mania-2025/WNCAATourneyDetailedResults.csv
/kaggle/input/march-machine-learning-mania-2025/WRegularSeasonCompactResults.csv
/kaggle/input/march-machine-learning-mania-2025/MNCAATourneySeedRoundSlots.csv
/kaggle/input/march-machine-learning-mania-2025/MRegularSeasonDetailedResults.csv
/kaggle/input/march-machine-learning-mania-2025/MNCAATourneyCompactResults.csv
/kaggle/input/march-machine-learning-mania-2025/MGameCities.csv
/kaggle/input/march-machine-learning-mania-2025/WSecondaryTourneyCompactResults.csv
/kaggle/input/march-machine-learning-mania-2025/WGameCities.csv
/kaggle/input/march-machine-learning-mania-2025/MSeasons.csv
/kaggle/input/march-machine-learning-mania-2025/WNCAATourneySlots.csv
/kaggle/input/march-machine-learning-mania-2025/MSecondaryTourneyTeams.csv
/kaggle/input/march-machine-learning-mania-20

In [6]:
import pandas as pd
import numpy as np
import itertools
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

#-------------------------------------------------------------------
# 1) LOAD THE DATA
#-------------------------------------------------------------------
# Adjust file paths as needed for your environment:
MTeams = pd.read_csv("/kaggle/input/march-machine-learning-mania-2025/MTeams.csv")
WTeams = pd.read_csv("/kaggle/input/march-machine-learning-mania-2025/WTeams.csv")
MSeasons = pd.read_csv("/kaggle/input/march-machine-learning-mania-2025/MSeasons.csv")
WSeasons = pd.read_csv("/kaggle/input/march-machine-learning-mania-2025/WSeasons.csv")

# Regular-season (compact) results – men’s
MRegSeasonCompact = pd.read_csv("/kaggle/input/march-machine-learning-mania-2025/MRegularSeasonCompactResults.csv")

# NCAA Tournament (compact) results – men’s
MNCAATourneyCompact = pd.read_csv("/kaggle/input/march-machine-learning-mania-2025/MNCAATourneyCompactResults.csv")

# Seeds – men’s
MNCAATourneySeeds = pd.read_csv("/kaggle/input/march-machine-learning-mania-2025/MNCAATourneySeeds.csv")

#-------------------------------------------------------------------
# 2) QUICK LOOK AT THE DATA
#-------------------------------------------------------------------
print("MTeams sample:")
print(MTeams.head(), "\n")

print("MSeasons sample:")
print(MSeasons.head(), "\n")

print("MRegularSeasonCompactResults sample:")
print(MRegSeasonCompact.head(), "\n")

#-------------------------------------------------------------------
# 3) SIMPLE EDA / FEATURE EXAMPLE
#    Illustrate how to create a small dataset for a quick model
#-------------------------------------------------------------------
# We’ll demonstrate a small pipeline using men’s regular-season results.
# Basic idea: treat each game as one observation for a binary classification.

# Create a label: 1 if Home/Winning Team was 'WTeamID', 0 otherwise
# But we actually want to represent the probability that "Team A beats Team B."
# For now, let's just do something very simplified, ignoring neutral sites, etc.

# We'll rename columns to keep them standardized:
df = MRegSeasonCompact.copy()
df['TeamA'] = df['WTeamID']
df['TeamB'] = df['LTeamID']
df['TeamA_Score'] = df['WScore']
df['TeamB_Score'] = df['LScore']

# The "target": TeamA won => 1, TeamA lost => 0
df['TeamA_Won'] = 1

# But we only have the scenario "TeamA is WTeamID." If you want
# both perspectives (TeamA vs. TeamB and TeamB vs. TeamA), you could
# re-stack the dataset or do other transformations. For simplicity,
# we keep it single-direction here.

# Simple feature: point differential
df['ScoreDiff'] = df['TeamA_Score'] - df['TeamB_Score']

# Let's define some minimal features:
features = ['ScoreDiff']  # obviously very naive
X = df[features]
y = df['TeamA_Won']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42)

# Fit a trivial logistic regression (again, very naive)
model = LogisticRegression()
model.fit(X_train, y_train)

train_acc = model.score(X_train, y_train)
test_acc = model.score(X_test, y_test)
print(f"Train Accuracy: {train_acc:.3f}")
print(f"Test  Accuracy: {test_acc:.3f}")

#-------------------------------------------------------------------
# 4) CREATE A SAMPLE SUBMISSION
#-------------------------------------------------------------------
# In 2025, you have to predict *every* possible matchup for men’s and women’s teams.
# For demonstration, let’s show how you’d generate matchups just for men’s TeamIDs.
# (In reality, you’d combine the men’s and women’s IDs and do them all.)

all_mens_teams = MTeams['TeamID'].unique()  # e.g., 1000-1999

# Sort them so we can do combinations where TeamA < TeamB
all_mens_teams = sorted(all_mens_teams)

matchup_rows = []
for A, B in itertools.combinations(all_mens_teams, 2):
    # -----------------------------------------------------
    # This is where you'd compute the probability that team A
    # would beat team B. For instance:
    #   p = my_model.predict_proba( feature_vector_for(A,B) )
    #
    # But we have not engineered a robust feature pipeline for A vs. B.
    # So we'll just put a placeholder probability (like 0.5).
    # -----------------------------------------------------
    p = 0.50
    row_id = f"2025_{A}_{B}"
    matchup_rows.append((row_id, p))

# Convert to a DataFrame
submission_df = pd.DataFrame(matchup_rows, columns=["ID", "Pred"])

# Quick preview
print(submission_df.head(10))

# Save to CSV
submission_df.to_csv("submission.csv", index=False)

print("\nSample submission file saved as submission.csv")


MTeams sample:
   TeamID     TeamName  FirstD1Season  LastD1Season
0    1101  Abilene Chr           2014          2025
1    1102    Air Force           1985          2025
2    1103        Akron           1985          2025
3    1104      Alabama           1985          2025
4    1105  Alabama A&M           2000          2025 

MSeasons sample:
   Season     DayZero RegionW    RegionX    RegionY    RegionZ
0    1985  10/29/1984    East       West    Midwest  Southeast
1    1986  10/28/1985    East    Midwest  Southeast       West
2    1987  10/27/1986    East  Southeast    Midwest       West
3    1988  11/02/1987    East    Midwest  Southeast       West
4    1989  10/31/1988    East       West    Midwest  Southeast 

MRegularSeasonCompactResults sample:
   Season  DayNum  WTeamID  WScore  LTeamID  LScore WLoc  NumOT
0    1985      20     1228      81     1328      64    N      0
1    1985      25     1106      77     1354      70    H      0
2    1985      25     1112      63     1223  

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 1